In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import json
import sys
import copy
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import DateOffset, BDay
from pandas.tseries.frequencies import to_offset
# from clarion import positions

from api.gui.base import View, Tabs, Store
from api.utils import parse_offset
from api.gui.market_monitor_h import CurvesViewer, Pivot, get_issuers_curves, get_bond_ref, get_bday, get_history_table, DatePicker, maturity_thresholds
from api.gui.max_diff_h import LowestFieldCurve
from api.gui.visual_h import Visual
from api.gui.fwd_pivot import PivotForward
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)

pd.options.plotting.backend = "plotly"

In [3]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
br = pd.read_csv(data_path / 'bonds_reference.csv')
bh = pd.read_csv(data_path / 'bonds_history_all.csv', parse_dates=['date'], dayfirst=True)

In [6]:
store = Store()
store.data['bond_ref'] = get_bond_ref()
store.data['bond_hist'] = copy.deepcopy(bh)
# store.data['issuers_curves'] = copy.deepcopy(issuers_curves)

In [17]:
app = Tabs(
    {
        'curves': CurvesViewer(build=False),
        'pivot': Pivot(build=False),
        'max diff': LowestFieldCurve(build=False),
        'visual': Visual(build=False),
        'fwd pivot': PivotForward(build=False)
    },
    store=store
)
app

Card(children=[Tabs(children=[Tab(children=['curves']), Tab(children=['pivot']), Tab(children=['max diff']), T…

In [ ]:
from itertools import combinations

In [ ]:
def get_fwd_z_spread(bonds, cols=None):
    
    bonds = bonds.sort_values('years_to_workout')
    cols = cols or [*bonds.columns]
    risk_ratio = bonds.iloc[0]['risk'] / bonds.iloc[1]['risk']
    fwd = (bonds.iloc[1]['z_spread'] - bonds.iloc[0]['z_spread'] * risk_ratio) / (1 - risk_ratio)
    name = f"{bonds.iloc[0]['maturity_label']}y{bonds.iloc[1]['maturity_label'] - bonds.iloc[0]['maturity_label']}y"
    fwd_series = bonds.iloc[0][cols]
    fwd_series['z_spread'] = fwd
    fwd_series['maturity_label'] = name
    return fwd_series

In [ ]:
bonds = group

In [ ]:
ticker = 'LEBAN'
currency = 'USD'
field = 'last_price'
source = 'BGN'
date = pd.to_datetime('20180430')

In [ ]:
dfr = br.query(f"ticker == '{ticker}' and currency == '{currency}'")
dfh = bh.query(f"security in {[*dfr['security']]} and source == '{source}'")
dfhd = dfh.set_index('date').loc[get_bday(date)].set_index('security')
gg = dfhd.loc[[dfhd[field].idxmin(), dfhd[field].idxmax()]].sort_values(by=field)
df_plot = dfh.query(f"security in {[*gg.index]}").pivot(index='date', columns='security', values=field)
df_plot = df_plot.assign(diff=df_plot.iloc[:, 1] - df_plot.iloc[:, 0])
dfrg = dfr.query(f"security in {[*gg.index]}")

In [ ]:
dfrg

In [ ]:
d1 = '2019-06-01'
d2 = '2020-06-06'

if not d1:
    d1 = df_plot.index[0]
if not d2:
    d2 = df_plot.index[-1]

In [ ]:
fig1 = df_plot['diff'].loc[d1:d2].dropna().plot().add_vline(x=date)
fig2 = df_plot.iloc[:, :2].dropna().loc[d1:d2].plot()

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

In [ ]:
for i, f in enumerate([fig1, fig2], 1):
    for trace in f.data:
        fig.add_trace(
            trace,
            row=i, col=1
        )


In [ ]:
fig.update_layout(height=1000)